### Dataset download

In [1]:
!gdown --id 159UNifFg4fN4HlvtiQEr_orCIWl68CJr

Downloading...
From: https://drive.google.com/uc?id=159UNifFg4fN4HlvtiQEr_orCIWl68CJr
To: /content/words.tgz
821MB [00:05, 156MB/s]


In [2]:
!mkdir words
!mv words.tgz words
%cd words
!tar -xvzf words.tgz

/content/words
/content/words


In [4]:
%cd ..

/content


In [5]:
!gdown --id 19YUCAX1o_GNv1zCsFk_RrMwyGuygs2qq

Downloading...
From: https://drive.google.com/uc?id=19YUCAX1o_GNv1zCsFk_RrMwyGuygs2qq
To: /content/data_file.zip
2.61MB [00:00, 41.1MB/s]


In [6]:
!unzip data_file.zip

Archive:  data_file.zip
   creating: data_file/
  inflating: data_file/lines.txt     
  inflating: data_file/sentences.txt  
  inflating: data_file/forms.txt     
  inflating: data_file/words.txt     


In [7]:
!mv data_file/words.txt ./

### import modules

In [8]:
import cv2
import keras.backend as K
import numpy as np
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, LSTM, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from tensorflow.python.client import device_lib

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


### Dataloader and utils

In [9]:
with open('./words.txt') as f:
    contents = f.readlines()
lines = [line.strip() for line in contents]
print(lines)

max_label_len = 0
char_list = "!\"#&'()*+,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

print(char_list, len(char_list))


def encode_to_labels(txt):
    # encoding each output word into digits
    dig_lst = []
    for index, chara in enumerate(txt):
        dig_lst.append(char_list.index(chara))

    return dig_lst


images = []
labels = []
RECORDS_COUNT = 10000

train_images = []
train_labels = []
train_input_length = []
train_label_length = []
train_original_text = []

valid_images = []
valid_labels = []
valid_input_length = []
valid_label_length = []
valid_original_text = []

inputs_length = []
labels_length = []


def process_image(img):
    """
    Converts image to shape (32, 128, 1) & normalize
    """
    w, h = img.shape

    # Aspect Ratio Calculation
    new_w = 32
    new_h = int(h * (new_w / w))
    img = cv2.resize(img, (new_h, new_w))
    w, h = img.shape

    img = img.astype('float32')

    # Converts each to (32, 128, 1)
    if w < 32:
        add_zeros = np.full((32 - w, h), 255)
        img = np.concatenate((img, add_zeros))
        w, h = img.shape

    if h < 128:
        add_zeros = np.full((w, 128 - h), 255)
        img = np.concatenate((img, add_zeros), axis=1)
        w, h = img.shape

    if h > 128 or w > 32:
        dim = (128, 32)
        img = cv2.resize(img, dim)

    img = cv2.subtract(255, img)

    img = np.expand_dims(img, axis=2)

    # Normalize
    img = img / 255

    return img


for index, line in enumerate(lines):
    splits = line.split(' ')
    status = splits[1]

    if status == 'ok':
        word_id = splits[0]
        word = "".join(splits[8:])

        splits_id = word_id.split('-')
        filepath = 'words/{}/{}-{}/{}.png'.format(splits_id[0],
                                                  splits_id[0],
                                                  splits_id[1],
                                                  word_id)

        # process image
        img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
        try:
            img = process_image(img)
        except:
            continue

        # process label
        try:
            label = encode_to_labels(word)
        except:
            continue

        if index % 10 == 0:
            valid_images.append(img)
            valid_labels.append(label)
            valid_input_length.append(31)
            valid_label_length.append(len(word))
            valid_original_text.append(word)
        else:
            train_images.append(img)
            train_labels.append(label)
            train_input_length.append(31)
            train_label_length.append(len(word))
            train_original_text.append(word)

        if len(word) > max_label_len:
            max_label_len = len(word)

    if index >= RECORDS_COUNT:
        break

train_padded_label = pad_sequences(train_labels,
                                   maxlen=max_label_len,
                                   padding='post',
                                   value=len(char_list))

valid_padded_label = pad_sequences(valid_labels,
                                   maxlen=max_label_len,
                                   padding='post',
                                   value=len(char_list))

print(train_padded_label.shape, valid_padded_label.shape)

train_images = np.asarray(train_images)
train_input_length = np.asarray(train_input_length)
train_label_length = np.asarray(train_label_length)

valid_images = np.asarray(valid_images)
valid_input_length = np.asarray(valid_input_length)
valid_label_length = np.asarray(valid_label_length)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



(7850, 16) (876, 16)


### Model CRNN

In [10]:
# input with shape of height=32 and width=128
inputs = Input(shape=(32, 128, 1))

# convolution layer with kernel size (3,3)
conv_1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
# poolig layer with kernel size (2,2)
pool_1 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_1)

conv_2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool_1)
pool_2 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_2)

conv_3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool_2)

conv_4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv_3)
# poolig layer with kernel size (2,1)
pool_4 = MaxPool2D(pool_size=(2, 1))(conv_4)

conv_5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool_4)
# Batch normalization layer
batch_norm_5 = BatchNormalization()(conv_5)

conv_6 = Conv2D(512, (3, 3), activation='relu', padding='same')(batch_norm_5)
batch_norm_6 = BatchNormalization()(conv_6)
pool_6 = MaxPool2D(pool_size=(2, 1))(batch_norm_6)

conv_7 = Conv2D(512, (2, 2), activation='relu')(pool_6)

squeezed = Lambda(lambda x: K.squeeze(x, 1))(conv_7)

# bidirectional LSTM layers with units=128
blstm_1 = Bidirectional(LSTM(256, return_sequences=True, dropout=0.2))(squeezed)
blstm_2 = Bidirectional(LSTM(256, return_sequences=True, dropout=0.2))(blstm_1)

outputs = Dense(len(char_list) + 1, activation='softmax')(blstm_2)

# model to be used at test time
act_model = Model(inputs, outputs)

the_labels = Input(name='the_labels', shape=[max_label_len], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')

### Loss CTC

In [11]:
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args

    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)


loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([outputs, the_labels, input_length, label_length])

### Train

In [12]:
model = Model(inputs=[inputs, the_labels, input_length, label_length], outputs=loss_out)

batch_size = 16
epochs = 100
e = str(epochs)
optimizer_name = 'adam'

model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=optimizer_name, metrics=['accuracy'])

filepath = "{}o-{}r-{}e-{}t-{}v.hdf5".format(optimizer_name,
                                             str(RECORDS_COUNT),
                                             str(epochs),
                                             str(train_images.shape[0]),
                                             str(valid_images.shape[0]))

checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
callbacks_list = [checkpoint]

# os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

history = model.fit(x=[train_images, train_padded_label, train_input_length, train_label_length],
                    y=np.zeros(len(train_images)),
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=([valid_images, valid_padded_label, valid_input_length, valid_label_length],
                                     [np.zeros(len(valid_images))]),
                    verbose=1, callbacks=callbacks_list)

model.save("text_recognize_weight.h5")

Epoch 1/100
491/491 [==============================] - 59s 41ms/step - loss: 17.4953 - accuracy: 0.0015 - val_loss: 12.9416 - val_accuracy: 0.0023

Epoch 00001: val_loss improved from inf to 12.94159, saving model to adamo-10000r-100e-7850t-876v.hdf5
Epoch 2/100
491/491 [==============================] - 17s 34ms/step - loss: 12.1178 - accuracy: 0.0452 - val_loss: 10.9308 - val_accuracy: 0.0753

Epoch 00002: val_loss improved from 12.94159 to 10.93080, saving model to adamo-10000r-100e-7850t-876v.hdf5
Epoch 3/100
491/491 [==============================] - 17s 34ms/step - loss: 10.2106 - accuracy: 0.0805 - val_loss: 9.6882 - val_accuracy: 0.1039

Epoch 00003: val_loss improved from 10.93080 to 9.68824, saving model to adamo-10000r-100e-7850t-876v.hdf5
Epoch 4/100
491/491 [==============================] - 17s 34ms/step - loss: 8.4380 - accuracy: 0.1282 - val_loss: 7.8132 - val_accuracy: 0.1724

Epoch 00004: val_loss improved from 9.68824 to 7.81316, saving model to adamo-10000r-100e-785

KeyboardInterrupt: ignored